In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
pd.set_option('display.max_columns', None)
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [25]:
# load in input files (diagnosis files)
inpatient_diagnosis = pd.read_csv('/content/drive/MyDrive/Data/Output/diagnosis.csv')
outpatient_diagnosis = pd.read_csv('/content/drive/MyDrive/Data/Output/outpatient_diagnosis.csv')
professional_diagnosis = pd.read_csv('/content/drive/MyDrive/Data/Output/professional_diagnosis.csv')

# CCSR lookup file for ICD10 regrouping
## look up was obtained from HCUP: https://hcup-us.ahrq.gov/toolssoftware/ccsr/dxccsr.jsp#download
LU_CCSR = pd.read_csv('/content/drive/MyDrive/Data/Output/DXCCSR_v2025-1_processed.csv')
LU_CCSR

,ICD_CD,CCSR_CATEGORY_CD,CCSR_CATEGORY_DESCRIPTION
0,A000,DIG001,Intestinal infection
1,A001,DIG001,Intestinal infection
2,A009,DIG001,Intestinal infection
3,A0100,DIG001,Intestinal infection
4,A0101,INF003,Bacterial infections
...,...,...,...
75233,Z9912,FAC012,Other specified encounters and counseling
75234,Z992,FAC025,Other specified status
75235,Z993,FAC025,Other specified status
75236,Z9981,FAC025,Other specified status


In [26]:
# prompt: for each of the three diagnosis files, create a column ICD_CD_TRUNC which is the first 3 characters of ICD_DIAG_CD
inpatient_diagnosis['ICD_CD_TRUNC'] = inpatient_diagnosis['ICD_DIAG_CD'].str[:3]
outpatient_diagnosis['ICD_CD_TRUNC'] = outpatient_diagnosis['ICD_DIAG_CD'].str[:3]
professional_diagnosis['ICD_CD_TRUNC'] = professional_diagnosis['ICD_DIAG_CD'].str[:3]

In [27]:
# prompt: Combine all three diagnosis files and create a dataframe of unique ICD_DIAG_CD only
# Combine all three diagnosis files, then get unique ICD_DIAG_CD
all_diagnosis = pd.concat([inpatient_diagnosis, outpatient_diagnosis, professional_diagnosis], ignore_index=True)
unique_icd_diag_cd = pd.DataFrame({'ICD_DIAG_CD': all_diagnosis['ICD_DIAG_CD'].unique()})
unique_icd_diag_cd

,ICD_DIAG_CD
0,S134XX
1,Z3480
2,T7432X
3,S8290X
4,Z3400
...,...
381,P269
382,K8020
383,R042
384,Z20822


In [28]:
# prompt: Let's merge LU_CCSR and unique_icd_diag_cd through ICD_CD=ICD_DIAG_CD. Are there any rows in unique_icd_diag_cd that have no corresponding values in CCSR_CATEGORY_DESCRIPTION?

# Merge LU_CCSR and unique_icd_diag_cd on ICD_CD=ICD_DIAG_CD
merged_df = pd.merge(unique_icd_diag_cd, LU_CCSR, left_on='ICD_DIAG_CD', right_on='ICD_CD', how='left')

# Check for rows in unique_icd_diag_cd with no corresponding values in CCSR_CATEGORY_DESCRIPTION
rows_without_match = merged_df[merged_df['CCSR_CATEGORY_DESCRIPTION'].isnull()]
print(len(rows_without_match))

50


In [29]:
# replace missing with UNK (for unknown)
merged_df['CCSR_CATEGORY_CD'] = merged_df['CCSR_CATEGORY_CD'].fillna('UNK')
merged_df['CCSR_CATEGORY_DESCRIPTION'] = merged_df['CCSR_CATEGORY_DESCRIPTION'].fillna('UNK')

# drop ICD_CD and CCSR_CATEGORY_CD
LU_CCSR_final = merged_df.drop(columns=['ICD_CD', 'CCSR_CATEGORY_CD'])
LU_CCSR_final

,ICD_DIAG_CD,CCSR_CATEGORY_DESCRIPTION
0,S134XX,UNK
1,Z3480,"Uncomplicated pregnancy, delivery or puerperium"
2,T7432X,UNK
3,S8290X,UNK
4,Z3400,"Uncomplicated pregnancy, delivery or puerperium"
...,...,...
381,P269,Respiratory perinatal condition
382,K8020,Biliary tract disease
383,R042,Respiratory signs and symptoms
384,Z20822,"Exposure, encounters, screening or contact wit..."


In [30]:
# merge diagnosis files with the CCSR LU file
inpatient_diagnosis = pd.merge(inpatient_diagnosis, LU_CCSR_final, on='ICD_DIAG_CD', how='left')
inpatient_diagnosis['CCSR_CATEGORY_DESCRIPTION'].isnull().sum()

np.int64(0)

In [31]:
outpatient_diagnosis = pd.merge(outpatient_diagnosis, LU_CCSR_final, on='ICD_DIAG_CD', how='left')
outpatient_diagnosis['CCSR_CATEGORY_DESCRIPTION'].isnull().sum()

np.int64(0)

In [32]:
professional_diagnosis = pd.merge(professional_diagnosis, LU_CCSR_final, on='ICD_DIAG_CD', how='left')
professional_diagnosis['CCSR_CATEGORY_DESCRIPTION'].isnull().sum()

np.int64(0)

In [33]:
inpatient_diagnosis

,BENE_ID,YR,ICD_DIAG_CD,ICD_Description,ICD_CD_TRUNC,CCSR_CATEGORY_DESCRIPTION
0,-10000010254618,2015,S134XX,"A Sprain of ligaments of cervical spine, initi...",S13,UNK
1,-10000010254653,2015,Z3480,Encounter for supervision of other normal preg...,Z34,"Uncomplicated pregnancy, delivery or puerperium"
2,-10000010254653,2017,T7432X,"A Child psychological abuse, confirmed, initia...",T74,UNK
3,-10000010254656,2017,S8290X,A Unspecified fracture of unspecified lower le...,S82,UNK
4,-10000010254656,2018,Z3480,Encounter for supervision of other normal preg...,Z34,"Uncomplicated pregnancy, delivery or puerperium"
...,...,...,...,...,...,...
176685,-10000010287573,2021,J189,"Pneumonia, unspecified organism",J18,Pneumonia (except that caused by tuberculosis)
176686,-10000010287573,2022,K219,Gastro-esophageal reflux disease without esoph...,K21,Esophageal disorders
176687,-10000010287584,2016,O039,Complete or unspecified spontaneous abortion w...,O03,Spontaneous abortion and complications of spon...
176688,-10000010287673,2018,D649,"Anemia, unspecified",D64,Aplastic anemia


In [34]:
# drop ICD_DIAG_CD and ICD_Description, then drop duplicates
inpatient_categories = inpatient_diagnosis.drop(columns=['ICD_DIAG_CD', 'ICD_Description']).drop_duplicates()
outpatient_categories = outpatient_diagnosis.drop(columns=['ICD_DIAG_CD', 'ICD_Description']).drop_duplicates()
professional_categories = professional_diagnosis.drop(columns=['ICD_DIAG_CD','ICD_Description']).drop_duplicates()

In [35]:
inpatient_categories

,BENE_ID,YR,ICD_CD_TRUNC,CCSR_CATEGORY_DESCRIPTION
0,-10000010254618,2015,S13,UNK
1,-10000010254653,2015,Z34,"Uncomplicated pregnancy, delivery or puerperium"
2,-10000010254653,2017,T74,UNK
3,-10000010254656,2017,S82,UNK
4,-10000010254656,2018,Z34,"Uncomplicated pregnancy, delivery or puerperium"
...,...,...,...,...
176685,-10000010287573,2021,J18,Pneumonia (except that caused by tuberculosis)
176686,-10000010287573,2022,K21,Esophageal disorders
176687,-10000010287584,2016,O03,Spontaneous abortion and complications of spon...
176688,-10000010287673,2018,D64,Aplastic anemia


In [36]:
# create flags to indicate which file categorization/regrouping comes from (useful in subsequent merger)
inpatient_categories['IP_FLAG'] = 'Y'
outpatient_categories['OP_FLAG'] = 'Y'
professional_categories['PR_FLAG'] = 'Y'

In [46]:
# Merge all three data files
master_categories = pd.merge(inpatient_categories, outpatient_categories,
                             on=['BENE_ID','YR','ICD_CD_TRUNC','CCSR_CATEGORY_DESCRIPTION'], how='outer')
master_categories['CCSR_CATEGORY_DESCRIPTION'].isnull().sum()

np.int64(0)

In [47]:
master_categories = pd.merge(master_categories, professional_categories,
                             on=['BENE_ID','YR','ICD_CD_TRUNC','CCSR_CATEGORY_DESCRIPTION'], how='outer')
master_categories['CCSR_CATEGORY_DESCRIPTION'].isnull().sum()

np.int64(0)

In [48]:
master_categories

,BENE_ID,YR,ICD_CD_TRUNC,CCSR_CATEGORY_DESCRIPTION,IP_FLAG,OP_FLAG,PR_FLAG
0,-10000010288010,2015,D64,Aplastic anemia,NaN,NaN,Y
1,-10000010288010,2015,E66,Obesity,NaN,NaN,Y
2,-10000010288010,2015,M54,Spondylopathies/spondyloarthropathy (including...,NaN,NaN,Y
3,-10000010288010,2015,P90,Neonatal cerebral disorders,NaN,NaN,Y
4,-10000010288010,2015,R73,Prediabetes,NaN,NaN,Y
...,...,...,...,...,...,...,...
834004,-10000010254618,2023,J32,Sinusitis,NaN,Y,NaN
834005,-10000010254618,2023,M79,"Musculoskeletal pain, not low back pain",NaN,Y,NaN
834006,-10000010254618,2023,N46,Male infertility,NaN,Y,NaN
834007,-10000010254618,2023,Z59,Socioeconomic/psychosocial factors,NaN,Y,NaN


In [49]:
# replace missing values in flag columns with 'N' for "no"
master_categories['IP_FLAG'] = master_categories['IP_FLAG'].fillna('N')
master_categories['OP_FLAG'] = master_categories['OP_FLAG'].fillna('N')
master_categories['PR_FLAG'] = master_categories['PR_FLAG'].fillna('N')

In [50]:
# prompt: count the number of rows by CCSR_CATEGORY_DESCRIPTION and ICD_CD_TRUNC
row_counts = master_categories.groupby(['CCSR_CATEGORY_DESCRIPTION', 'ICD_CD_TRUNC'])['BENE_ID'].count().reset_index(name='Row_Count')
row_counts

,CCSR_CATEGORY_DESCRIPTION,ICD_CD_TRUNC,Row_Count
0,Abdominal pain and other digestive/abdomen sig...,R13,194
1,Abdominal pain and other digestive/abdomen sig...,R19,130
2,Abnormal findings without diagnosis,R73,3750
3,Abnormal findings without diagnosis,R76,17
4,Abnormal findings without diagnosis,R93,22136
...,...,...,...
242,Urinary tract infections,N30,681
243,Urinary tract infections,N39,527
244,Viral infection,B08,1784
245,Viral infection,B34,3507


In [51]:
# prompt: count the number of "UNK" under CCSR_CATEGORY_DESCRIPTION within the master_categories data
unk_count = master_categories[master_categories['CCSR_CATEGORY_DESCRIPTION'] == 'UNK'].shape[0]
print(f"The number of 'UNK' under CCSR_CATEGORY_DESCRIPTION is: {unk_count}")

The number of 'UNK' under CCSR_CATEGORY_DESCRIPTION is: 48930


* Too many unknowns (ie., UNK; $n=48930$).
* Most of the 3-character truncated codes range between S00 and T88, mainly corresponding to injury or poisoning, among others.
* If we can replace UNKs correspinding to these truncated codes with injury/poissoning, then we can reduce the number of UNKs overall

In [52]:
# prompt: if ICD_CD_TRUNC in master_categories is in the range S00-T88 and CCSR_CATEGORY_DESCRIPTION is UNK, then recode CCSR_CATEGORY_DESCRIPTION as "Injury or poisoning and others"

# Create a boolean mask for rows where ICD_CD_TRUNC is within the range S00-T88 and CCSR_CATEGORY_DESCRIPTION is UNK
mask = (master_categories['ICD_CD_TRUNC'].between('S00', 'T88', inclusive="both")) & (master_categories['CCSR_CATEGORY_DESCRIPTION'] == 'UNK')

# Recode CCSR_CATEGORY_DESCRIPTION for the rows matching the mask
master_categories.loc[mask, 'CCSR_CATEGORY_DESCRIPTION'] = 'Injury or poisoning and others'

In [54]:
row_counts = master_categories.groupby(['CCSR_CATEGORY_DESCRIPTION'])['BENE_ID'].count().reset_index(name='Row_Count')
row_counts

,CCSR_CATEGORY_DESCRIPTION,Row_Count
0,Abdominal pain and other digestive/abdomen sig...,324
1,Abnormal findings without diagnosis,25903
2,Acquired absence of limb or organ,3596
3,Acute and chronic tonsillitis,2120
4,Acute bronchitis,4271
...,...,...
135,Trauma- and stressor-related disorders,11
136,UNK,631
137,"Uncomplicated pregnancy, delivery or puerperium",2056
138,Urinary tract infections,1632


Only 631 unknowns now! Bingo!!

In [56]:
master_categories.to_csv('/content/drive/MyDrive/Data/Output/master_categories.csv', index=False)